In [63]:
import pymongo
import streamlit as st
import pandas as pd

def connect_to_mongodb():
    secrets = st.secrets["mongo"]
    conn_str = secrets["conn_str"]
    client = pymongo.MongoClient(conn_str)
    db_name = client.ClimbingGradeFeedback
    collection_name = db_name.ClimbingFeedbackStreamlit

    return collection, client

# Test the connection
try:
    collection, client = connect_to_mongodb()
    # The ismaster command is cheap and does not require auth.
    client.admin.command('ismaster')
    print("MongoDB connection successful")
except Exception as e:
    print("MongoDB connection unsuccessful")
    print(e)

MongoDB connection successful


In [64]:
# Run the function to connect to MongoDB and get the cursor
collection, client = connect_to_mongodb()

# Query the MongoDB collection and get a cursor
cursor = collection.find()

# Create a copy of the cursor for later use
cursor_copy = cursor.clone()

In [65]:
# Convert the original cursor (cursor_copy) to a list and then create a DataFrame
df = pd.DataFrame(list(cursor_copy))

In [71]:
# print the spread of values in the actual_sport_grade column and actual_boulder_grade column
# this will help us to see if the data is clean, and if there are any values that need to be removed or changed
print(df.actual_sport_grade.value_counts())


actual_sport_grade
0                                      6
6c                                     4
6b                                     4
7c+                                    3
7b                                     3
6b                                     3
6c+                                    3
5.11                                   2
7a                                     2
8a                                     2
7b+                                    2
6b+                                    2
6a                                     2
Na                                     2
5.13                                   2
5.12d                                  1
8c+                                    1
7a+                                    1
N/A (like 11b, i don't sport climb)    1
8b                                     1
12.b                                   1
28                                     1
5.12c                                  1
7A+                                   

In [ ]:
print(df.actual_bouldering_grade.value_counts())

In [66]:
# based on the above results, clean the data as needed
df.fillna(0, inplace=True)
df.replace('', 0, inplace=True)
df.replace('Na', 0, inplace=True)

In [67]:
df.head()

,_id,strength_to_weight_pullup,strength_to_weight_maxhang,strength_to_weight_weightpull,continuous,repeaters1,exp,trainexp,days,predicted_bouldering_grade,actual_bouldering_grade,predicted_sport_grade,actual_sport_grade,height,weight
0,653dffec24d1021354ce9a87,1.4,1.4,1.4,30.0,120.0,5.0,3.0,8.0,V5,V7,7a+,7c+,0,0
1,653e002024d1021354ce9a89,1.4,1.4,1.4,30.0,120.0,5.0,3.0,8.0,V5,V7,7a+,7b,0,0
2,653e050724d1021354ce9a8b,2.0256410256410255,3.5641025641025643,3.6666666666666665,200.0,1000.0,1.0,1.0,1.0,V10,v8,7a+,0,0,0
3,653e0bdd24d1021354ce9a8d,1.1310043668122272,1.0,1.0,2.0,10.0,1.0,1.0,0.0,V4,V4,5c,6a,0,0
4,653e0cc224d1021354ce9a8f,1.4,1.4,1.4,30.0,120.0,5.0,3.0,8.0,V5,V8,7a+,7b+,0,0


In [68]:
print(df.columns)
print(len(df))

Index(['_id', 'strength_to_weight_pullup', 'strength_to_weight_maxhang',
       'strength_to_weight_weightpull', 'continuous', 'repeaters1', 'exp',
       'trainexp', 'days', 'predicted_bouldering_grade',
       'actual_bouldering_grade', 'predicted_sport_grade',
       'actual_sport_grade', 'height', 'weight'],
      dtype='object')
51


In [69]:
# remove the '_id' and predicted fields field and rename columns
new_data_mongodb = df.drop(['_id','predicted_bouldering_grade','predicted_sport_grade','height','weight'], axis=1, inplace=True)
new_data_mongodb = df.rename(columns={
    'actual_bouldering_grade': 'max_boulder_grade',
    'actual_sport_grade': 'max_sport_grade'
})

In [ ]:
# save the new data to include in the models
new_data_mongodb.to_csv('new_data_mongodb.csv', index=False)